In [ ]:
import tensorflow as tf
import os

In [ ]:
tf.__version__

In [ ]:
# Enable memory growth (prevents TensorFlow from pre-allocating all VRAM)

from tensorflow.python.client import device_lib

gpus = tf.config.experimental.list_physical_devices('GPU')

print("Num GPUs Available:", len(gpus))
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Built with GPU support:", tf.test.is_built_with_gpu_support())
print(device_lib.list_local_devices())

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
# Enable memory growth (prevents TensorFlow from pre-allocating all VRAM)

gpus = tf.config.experimental.list_physical_devices('GPU')

print("Num of GPUs", len(gpus))

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

**load the data**

In [ ]:
currentDir = os.getcwd()

Train_filePath = os.path.join(currentDir, "Flower Classification Dataset/train")
Test_filePath = os.path.join(currentDir, "Flower Classification Dataset/test")
Validation_filePath = os.path.join(currentDir, "Flower Classification Dataset/valid")

Train_data = tf.keras.preprocessing.image_dataset_from_directory(
    Train_filePath,
    labels = 'inferred',
    image_size = (224,224),
    batch_size = 102,
    #validation_split = 0
)

Test_data = tf.keras.preprocessing.image_dataset_from_directory(
    Test_filePath,
    labels = 'inferred',
    image_size = (224,224),
    batch_size = 102,
    #validation_split = 0
)

Validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    Validation_filePath,
    labels = 'inferred',
    image_size = (224,224),
    batch_size = 102,
    #validation_split = 0
)

#Split the dataset into input and target
# Input_validation= Validation_data.map(lambda image, lable: image)
# Target_validation= Validation_data.map(lambda image, lable: lable)

**Outline the model**

In [ ]:
input_size = 50176
output_size = 102
hidden_layer_size = 128 #TODO: figure out the exact value

model = tf.keras.Sequential([
    # ReScale pixel values in the band [0, 1] 
    tf.keras.Input(shape=(224, 224, 3)),
    tf.keras.layers.Rescaling(1./255),

    # Augmentation (Adding randomization to the images)
    # ONLY USED IN TRAINING
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.2),
    
    # Conv 1
    tf.keras.layers.Conv2D(
        32, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ), #TODO: Check if the combination of L1 and L2 works better for all layers
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 2
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 3
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 4
    tf.keras.layers.Conv2D(
        128, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    # Flattened Inputs
    tf.keras.layers.GlobalAveragePooling2D(), # Better than normal Flattening, reduces the dimensionality of feature maps
        #tf.keras.layers.Flatten(),
    
    # Dense Layers (Fully connected layers (FC Layers))
    tf.keras.layers.Dense(
        hidden_layer_size, 
        activation = 'relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    #TODO: Check if dropping neurons actually prevents overfitting 
    #TODO: We can test other techniques besides dropping 
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(        
        hidden_layer_size, 
        activation = 'relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.Dropout(0.3),

    # Didn't specify "softmax" activation function in the last layer 
    # to use the right order of opertions for loss calculations
    tf.keras.layers.Dense(output_size) 
])

model.summary()

**Choose the optimizer and the loss function**

In [ ]:
lossFunction = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) # Internally applies "softmax" if "from_logits = True"

#TODO: Add early stopping
model.compile(optimizer= 'adam' ,loss=lossFunction, metrics= ['accuracy'])

**Training**

In [ ]:
epoch_num= 10

model.fit(Train_data, validation_data= Validation_data, epochs= epoch_num, verbose= 2)

**Evaluate the model on test data**

In [ ]:
# test_loss, test_accuracy = model.evaluate(Test_data)
evaluation = model.evaluate(Test_data)
print(f"Evaluation: {evaluation}")
# print(f"Test accuracy: {test_accuracy}")